<div style="color: black; font-family: 'Times New Roman', serif; text-transform: uppercase; font-weight: bold; font-size: 24px; word-spacing: 10px;">
Mycobacterium Tuberculosis Detection Using CNN
</div>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2 as cv
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import os

In [ ]:
normaldir = '/content/drive/MyDrive/Myco/Normal'
tbdir = '/content/drive/MyDrive/Myco/Tuberculosis'
images = []
labels = []
imagesize = 256

In [ ]:
for x in os.listdir(normaldir):
    imagedir = os.path.join(normaldir, x)
    image = cv.imread(imagedir, cv.IMREAD_GRAYSCALE)
    image = cv.resize(image, (imagesize, imagesize))
    images.append(image)
    labels.append(0)

for y in os.listdir(tbdir):
    imagedir = os.path.join(tbdir, y)
    image = cv.imread(imagedir, cv.IMREAD_GRAYSCALE)
    image = cv.resize(image, (imagesize, imagesize))
    images.append(image)
    labels.append(1)

In [ ]:
images = np.array(images)
labels = np.array(labels)

#Splitting the images and labels into training and testing sets, then normalizing the values within them for computational efficiency (from 0-255 scale to 0-1 scale)
imagetrain, imagetest, labeltrain, labeltest = train_test_split(images, labels, test_size=0.3, random_state=42)
imagetrain = (imagetrain.astype('float32'))/255
imagetest = (imagetest.astype('float32'))/255

In [ ]:
print(imagetrain.shape)  # This will give you the current shape


(354, 256, 256)


In [ ]:
import numpy as np
from imblearn.over_sampling import SMOTE

# Example shape
imagetrain = np.random.rand(354, 256, 256)  # Replace with your actual data
labeltrain = np.random.randint(0, 2, 354)  # Example labels (binary)

# Flatten the images
imagetrain = imagetrain.reshape(354, 256 * 256)

# Apply SMOTE
smote = SMOTE(random_state=42)
imagetrain, labeltrain = smote.fit_resample(imagetrain, labeltrain)

# Reshape back to original image dimensions
imagetrain = imagetrain.reshape(-1, 256, 256, 1)

# Convert grayscale images to RGB for models
imagetrain = np.repeat(imagetrain, 3, axis=-1)

# Print the final shape
print(imagetrain.shape)  # Should reflect the new shape after processing


(378, 256, 256, 3)


In [ ]:
print(np.unique(labeltrain, return_counts=True))

(array([0, 1]), array([189, 189]))


## CNN

In [ ]:
import numpy as np
from keras import layers, models
from keras.callbacks import ReduceLROnPlateau

# Assuming imagetrain and labeltrain are already defined

# Check shapes
print("Image shape:", imagetrain.shape)  # Expected: (378, 256, 256, 3)
print("Label shape:", labeltrain.shape)   # Expected: (378,)

# Define the CNN model for RGB images
cnn = models.Sequential([
    layers.Input(shape=(256, 256, 3)),  # Now accepting RGB input
    layers.Conv2D(16, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
cnn.compile(
    loss='binary_crossentropy',
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

# Set up the learning rate reduction callback
reduce_lr = ReduceLROnPlateau(monitor='accuracy', factor=0.1, patience=1, min_lr=0.00001, verbose=1)

# Fit the model
cnn.fit(imagetrain, labeltrain, batch_size=16, epochs=10, verbose=2, callbacks=[reduce_lr])


Image shape: (378, 256, 256, 3)
Label shape: (378,)
Epoch 1/10
24/24 - 12s - 483ms/step - accuracy: 0.4683 - loss: 0.8467 - learning_rate: 0.0010
Epoch 2/10
24/24 - 0s - 19ms/step - accuracy: 0.5053 - loss: 0.6931 - learning_rate: 0.0010
Epoch 3/10
24/24 - 0s - 18ms/step - accuracy: 0.5132 - loss: 0.6945 - learning_rate: 0.0010
Epoch 4/10

Epoch 4: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
24/24 - 1s - 26ms/step - accuracy: 0.5079 - loss: 0.6932 - learning_rate: 0.0010
Epoch 5/10
24/24 - 0s - 18ms/step - accuracy: 0.5397 - loss: 0.6925 - learning_rate: 1.0000e-04
Epoch 6/10
24/24 - 0s - 19ms/step - accuracy: 0.5608 - loss: 0.6902 - learning_rate: 1.0000e-04
Epoch 7/10

Epoch 7: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
24/24 - 1s - 26ms/step - accuracy: 0.5370 - loss: 0.6925 - learning_rate: 1.0000e-04
Epoch 8/10
24/24 - 0s - 18ms/step - accuracy: 0.5794 - loss: 0.6910 - learning_rate: 1.0000e-05
Epoch 9/10

Epoch 9: ReduceLROnPlateau

**InceptionV3**

In [ ]:
from keras.applications import InceptionV3
from keras import layers, models

# Load the InceptionV3 model with pre-trained weights
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze the base model
base_model.trainable = False

# Create the new model
inception_model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
inception_model.compile(
    loss='binary_crossentropy',
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

# Fit the model
inception_model.fit(imagetrain, labeltrain, batch_size=16, epochs=10, verbose=2, callbacks=[reduce_lr])


87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/10
24/24 - 31s - 1s/step - accuracy: 0.4841 - loss: 0.7732 - learning_rate: 0.0010
Epoch 2/10
24/24 - 20s - 814ms/step - accuracy: 0.5397 - loss: 0.7089 - learning_rate: 0.0010
Epoch 3/10

Epoch 3: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
24/24 - 1s - 47ms/step - accuracy: 0.5238 - loss: 0.6929 - learning_rate: 0.0010
Epoch 4/10
24/24 - 1s - 46ms/step - accuracy: 0.5503 - loss: 0.6806 - learning_rate: 1.0000e-04
Epoch 5/10

Epoch 5: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
24/24 - 1s - 43ms/step - accuracy: 0.5238 - loss: 0.6913 - learning_rate: 1.0000e-04
Epoch 6/10
24/24 - 1s - 44ms/step - accuracy: 0.5556 - loss: 0.6870 - learning_rate: 1.0000e-05
Epoch 7/10

Epoch 7: ReduceLROnPlateau reducing learning rate to 1e-05.
24/24 - 1s - 44ms/step - accuracy: 0.5503 - loss: 0.6873 - learning_rate: 1.0000e-05
Epoch 8/10
24/24 - 1s - 44ms/step - accuracy: 0.5238 - loss: 0.6891 - l

**ResNet50**

In [ ]:
from keras.applications import ResNet50

# Load the ResNet50 model with pre-trained weights
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze the base model
base_model.trainable = False

# Create the new model
resnet_model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
resnet_model.compile(
    loss='binary_crossentropy',
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

# Fit the model
resnet_model.fit(imagetrain, labeltrain, batch_size=16, epochs=10, verbose=2, callbacks=[reduce_lr])


Epoch 1/10
24/24 - 20s - 837ms/step - accuracy: 0.4921 - loss: 0.7968 - learning_rate: 0.0010
Epoch 2/10
24/24 - 8s - 340ms/step - accuracy: 0.5212 - loss: 0.6956 - learning_rate: 0.0010
Epoch 3/10
24/24 - 1s - 55ms/step - accuracy: 0.5476 - loss: 0.6955 - learning_rate: 0.0010
Epoch 4/10

Epoch 4: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
24/24 - 2s - 102ms/step - accuracy: 0.5476 - loss: 0.6916 - learning_rate: 0.0010
Epoch 5/10

Epoch 5: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
24/24 - 1s - 54ms/step - accuracy: 0.5026 - loss: 0.7006 - learning_rate: 1.0000e-04
Epoch 6/10

Epoch 6: ReduceLROnPlateau reducing learning rate to 1e-05.
24/24 - 1s - 53ms/step - accuracy: 0.4894 - loss: 0.6956 - learning_rate: 1.0000e-05
Epoch 7/10
24/24 - 1s - 53ms/step - accuracy: 0.5106 - loss: 0.6942 - learning_rate: 1.0000e-05
Epoch 8/10
24/24 - 1s - 55ms/step - accuracy: 0.4709 - loss: 0.7050 - learning_rate: 1.0000e-05
Epoch 9/10
24/24 - 1s - 55m

**Xception**

In [ ]:
from keras.applications import Xception

# Load the Xception model with pre-trained weights
base_model = Xception(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze the base model
base_model.trainable = False

# Create the new model
xception_model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
xception_model.compile(
    loss='binary_crossentropy',
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

# Fit the model
xception_model.fit(imagetrain, labeltrain, batch_size=16, epochs=10, verbose=2, callbacks=[reduce_lr])


83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/10
24/24 - 24s - 981ms/step - accuracy: 0.5476 - loss: 0.6913 - learning_rate: 0.0010
Epoch 2/10

Epoch 2: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
24/24 - 2s - 72ms/step - accuracy: 0.5106 - loss: 0.7071 - learning_rate: 0.0010
Epoch 3/10

Epoch 3: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
24/24 - 2s - 73ms/step - accuracy: 0.5079 - loss: 0.6986 - learning_rate: 1.0000e-04
Epoch 4/10

Epoch 4: ReduceLROnPlateau reducing learning rate to 1e-05.
24/24 - 2s - 73ms/step - accuracy: 0.5238 - loss: 0.6887 - learning_rate: 1.0000e-05
Epoch 5/10
24/24 - 3s - 106ms/step - accuracy: 0.4841 - loss: 0.7034 - learning_rate: 1.0000e-05
Epoch 6/10
24/24 - 2s - 76ms/step - accuracy: 0.5317 - loss: 0.6948 - learning_rate: 1.0000e-05
Epoch 7/10
24/24 - 2s - 104ms/step - accuracy: 0.5238 - loss: 0.6960 - learning_rate: 1.0000e-05
Epoch 8/10
24/24 - 2s - 73ms/step - accuracy: 0.5026 - loss: 0.7

In [ ]:
import cv2 as cv
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import os
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

normaldir = '/content/drive/MyDrive/Myco/Normal'
tbdir = '/content/drive/MyDrive/Myco/Tuberculosis'
images = []
labels = []
imagesize = 224

for x in os.listdir(normaldir):
    imagedir = os.path.join(normaldir, x)
    image = cv.imread(imagedir, cv.IMREAD_GRAYSCALE)
    image = cv.resize(image, (imagesize, imagesize))
    image = np.expand_dims(image, axis=-1)
    images.append(image)
    labels.append(0)

for y in os.listdir(tbdir):
    imagedir = os.path.join(tbdir, y)
    image = cv.imread(imagedir, cv.IMREAD_GRAYSCALE)
    image = cv.resize(image, (imagesize, imagesize))
    image = np.expand_dims(image, axis=-1)
    images.append(image)
    labels.append(1)

images = np.array(images)
labels = np.array(labels)

imagetrain, imagetest, labeltrain, labeltest = train_test_split(images, labels, test_size=0.3, random_state=42)

imagetrain = (imagetrain.astype('float32')) / 255
imagetest = (imagetest.astype('float32')) / 255

imagetrain_rgb = np.repeat(imagetrain, 3, axis=-1)
imagetest_rgb = np.repeat(imagetest, 3, axis=-1)

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(imagesize, imagesize, 3))
base_model.trainable = False

x = base_model.output
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(learning_rate=0.001),
    metrics=['accuracy']
)

reduce_lr = ReduceLROnPlateau(monitor='accuracy', factor=0.1, patience=1, min_lr=0.00001, verbose=1)

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(imagetrain_rgb, labeltrain, batch_size=16)
test_generator = test_datagen.flow(imagetest_rgb, labeltest, batch_size=16)

model.fit(train_generator, epochs=10, verbose=2, callbacks=[reduce_lr])

print('TESTING DATA:')
model.evaluate(test_generator, verbose=2)

print('ADVANCED TESTING METRICS:')
from sklearn.metrics import classification_report, confusion_matrix

predictions = model.predict(test_generator, batch_size=16)
predicted_labels = (predictions > 0.5).astype('int32')

print(classification_report(labeltest, predicted_labels))
print(confusion_matrix(labeltest, predicted_labels))

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


23/23 - 14s - 595ms/step - accuracy: 0.5028 - loss: 6.6912 - learning_rate: 0.0010
Epoch 2/10

Epoch 2: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
23/23 - 2s - 95ms/step - accuracy: 0.5000 - loss: 0.7301 - learning_rate: 0.0010
Epoch 3/10

Epoch 3: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
23/23 - 1s - 58ms/step - accuracy: 0.5000 - loss: 0.6932 - learning_rate: 1.0000e-04
Epoch 4/10

Epoch 4: ReduceLROnPlateau reducing learning rate to 1e-05.
23/23 - 1s - 32ms/step - accuracy: 0.5000 - loss: 0.6931 - learning_rate: 1.0000e-05
Epoch 5/10
23/23 - 1s - 31ms/step - accuracy: 0.5000 - loss: 0.6931 - learning_rate: 1.0000e-05
Epoch 6/10
23/23 - 1s - 61ms/step - accuracy: 0.5000 - loss: 0.6931 - learning_rate: 1.0000e-05
Epoch 7/10
23/23 - 1s - 50ms/step - accuracy: 0.5000 - loss: 0.6931 - learning_rate: 1.0000e-05
Epoch 8/10
23/23 - 1s - 54ms/step - accuracy: 0.5000 - loss: 0.6931 - learning_rate: 1.0000e-05
Epoch 9/10
23/23 - 1s - 28ms/ste

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
